In [1]:
import anndata
import pandas as pd
from pathlib import Path

    Laughney肺癌和Lee结直肠癌用于训练。泰洛什头颈癌是用来检测的。在这里，我们使用带注释的数据矩阵(有关详细信息，请参阅AnnData)。

    adata.X	矩阵	numpy矩阵
    adata.obs	观测量	pandas Dataframe
    adata.var	特征量	pandas Dataframe
    adata.uns	非结构化数据	字典dict

In [2]:
paths = [
    Path("data/laughney20_lung/"),
    Path("data/lee20_crc/"),
    Path("data/tirosh17_headneck/")
]
names = [
    "laughney",
    "lee",
    "tirosh"
]
adatas = {}
for path, name in zip(paths, names):
    adatas[name] = anndata.read_h5ad(path / "adata.h5ad")
    # Uncomment to perform preprocessing. Here, the loaded anndata objects are already preprocessed. 
    # adatas[name] = data.preprocess_adata(adatas[name])

### 数据分析

In [3]:
adatas['laughney'].X[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
data = adatas['laughney'].obs.astype('str')
data

,raw,major,major_hallmark_corrected,tier_0,tier_1,tier_2,tier_3,tier_0_hallmark_corrected
0,Tm,Tumor,Immune,Tumor,Immune,Lymphoid,T,Normal
1,MACROPHAGE,Tumor,Immune,Tumor,Immune,Myeloid,Macrophage,Normal
2,DENDRITIC,Tumor,Immune,Tumor,Immune,Myeloid,Dendritic,Normal
3,Tm,Tumor,Immune,Tumor,Immune,Lymphoid,T,Normal
4,Tm,Tumor,Immune,Tumor,Immune,Lymphoid,T,Normal
...,...,...,...,...,...,...,...,...
40500,Tm,Tumor,Immune,Tumor,Immune,Lymphoid,T,Normal
40501,Tm,Tumor,Immune,Tumor,Immune,Lymphoid,T,Normal
40502,Tm,Tumor,Immune,Tumor,Immune,Lymphoid,T,Normal
40503,Tm,Tumor,Immune,Tumor,Immune,Lymphoid,T,Normal


In [11]:
#lets do Lable enconding coding to make more features 

from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
le_count = 0
# Iterate through the columns
for col in data:
    if data[col].dtype == 'object':
        # Train on the training data
        le.fit(data[col])
        # Transform both training and testing data
        data[col] = le.transform(data[col])
        
        
        # Keep track of how many columns were label encoded
        le_count += 1
rawdata = data           
print('%d columns were label encoded.' % le_count)

8 columns were label encoded.


In [12]:
data

,raw,major,major_hallmark_corrected,tier_0,tier_1,tier_2,tier_3,tier_0_hallmark_corrected
0,18,6,3,1,3,0,6,0
1,7,6,3,1,3,1,2,0
2,1,6,3,1,3,1,1,0
3,18,6,3,1,3,0,6,0
4,18,6,3,1,3,0,6,0
...,...,...,...,...,...,...,...,...
40500,18,6,3,1,3,0,6,0
40501,18,6,3,1,3,0,6,0
40502,18,6,3,1,3,0,6,0
40503,18,6,3,1,3,0,6,0


In [15]:
data.tier_0_hallmark_corrected.value_counts()

tier_0_hallmark_corrected
0    39414
1     1091
Name: count, dtype: int64

In [13]:
import pandas as pd
import xgboost as xgb

dataset = data

# 将特征和标签分离
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

# 划分训练集和测试集
train_size = int(len(X) * 0.8)
train_X, train_y = X[:train_size], y[:train_size]
test_X, test_y = X[train_size:], y[train_size:]

# 构建XGBoost模型
model = xgb.XGBClassifier()

# 拟合模型
model.fit(train_X, train_y)

# 进行预测
predictions = model.predict(test_X)

# 输出结果
print(predictions)


[0 0 0 ... 0 0 0]


In [17]:
# 计算准确率
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_y, predictions)

print("准确率：", accuracy)

准确率： 1.0


In [18]:
test_y.sum()

24

In [19]:
predictions.sum()

24

In [27]:
data

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [84]:
import torch
from torch.utils.data import WeightedRandomSampler

# 构造样本数据，一共11000个样本，正类样本1000个，负类样本10000个
data =  torch.cat([torch.ones(1000,10), torch.zeros(10000,10)])
labels = torch.cat([torch.ones(1000), torch.zeros(10000)])

# 计算每个样本的权重
class_sample_count = torch.Tensor([(labels == t).sum() for t in torch.unique(labels, sorted=True)])
weight = 1. / class_sample_count

# 将每个样本的权重与其对应的类别编号组合成一个样本-权重对
samples_weight = torch.zeros(len(labels))
for idx, label in enumerate(labels):
    samples_weight[idx] = weight[int(label)]

# 使用WeightedRandomSampler创建采样器
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

# 使用采样器来创建数据加载器
loader = torch.utils.data.DataLoader(dataset, batch_size=32, sampler=sampler)


In [85]:
for xx,yy in loader:
    print(yy)
    break

tensor([1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
        1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.])


In [86]:
import torch
from torch.utils.data import WeightedRandomSampler

# 构造样本数据，一共11000个样本，正类样本1000个，负类样本10000个
data =  torch.cat([torch.ones(1000,10), torch.zeros(10000,10),torch.zeros(10000,10)])
labels = torch.cat([torch.ones(1000), torch.zeros(10000),torch.Tensor(10000*[2])])

# 计算每个样本的权重
class_sample_count = torch.Tensor([(labels == t).sum() for t in torch.unique(labels, sorted=True)])
weights = [1.0 / class_sample_count[i] if i==0 else 10.0 / class_sample_count[i] for i in range(len(class_sample_count))]
weights = torch.DoubleTensor(weights)
samples_weight = weights[labels.long()]

# 使用WeightedRandomSampler创建采样器
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

# 使用采样器来创建数据加载器
dataset = torch.utils.data.TensorDataset(data, labels)
loader = torch.utils.data.DataLoader(dataset, batch_size=100, sampler=sampler)


In [87]:
for xx,yy in loader:
    print(yy)
    break

tensor([2., 1., 2., 1., 2., 1., 1., 2., 1., 1., 2., 2., 1., 2., 2., 2., 1., 2.,
        2., 2., 2., 2., 2., 2., 2., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
        2., 1., 2., 2., 2., 1., 0., 2., 0., 1., 2., 2., 0., 2., 2., 2., 1., 2.,
        1., 2., 1., 1., 1., 1., 2., 1., 0., 1., 1., 2., 2., 2., 2., 0., 2., 2.,
        0., 2., 2., 1., 1., 1., 2., 2., 1., 0., 1., 1., 1., 1., 2., 1., 1., 2.,
        2., 2., 2., 1., 1., 2., 2., 2., 2., 1.])


### 划分训练测试

In [89]:
import torch
from torch.utils.data import WeightedRandomSampler, TensorDataset, random_split

# 构造样本数据，一共11000个样本，正类样本1000个，负类样本10000个
data =  torch.cat([torch.ones(1000,10), torch.zeros(10000,10),torch.zeros(10000,10)])
labels = torch.cat([torch.ones(1000), torch.zeros(10000),torch.Tensor(10000*[2])])

# 计算每个样本的权重
class_sample_count = torch.Tensor([(labels == t).sum() for t in torch.unique(labels, sorted=True)])
weights = [1.0 / class_sample_count[i] if i==0 else 10.0 / class_sample_count[i] for i in range(len(class_sample_count))]
weights = torch.DoubleTensor(weights)
samples_weight = weights[labels.long()]

# 使用WeightedRandomSampler创建采样器
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

# 创建数据集
dataset = TensorDataset(data, labels)

# 划分训练集和测试集
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 使用采样器来创建训练集的数据加载器
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, sampler=sampler)

# 创建测试集的数据加载器
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True)


In [90]:
for xx,yy in loader:
    print(yy)
    break

tensor([2., 0., 1., 0., 2., 2., 2., 1., 1., 1., 2., 2., 1., 1., 1., 0., 2., 1.,
        2., 1., 1., 2., 1., 2., 2., 2., 1., 2., 2., 2., 1., 2., 2., 1., 1., 2.,
        2., 2., 2., 2., 1., 2., 1., 2., 2., 2., 1., 2., 1., 2., 1., 2., 2., 1.,
        2., 2., 2., 1., 1., 2., 1., 2., 2., 1., 1., 0., 2., 2., 2., 2., 2., 1.,
        1., 2., 1., 1., 0., 1., 1., 1., 1., 1., 1., 2., 0., 1., 2., 1., 1., 1.,
        1., 1., 1., 2., 2., 1., 1., 1., 1., 2.])
